In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.__version__,np.__version__

('0.24.1', '1.16.2')

In [3]:
df = pd.DataFrame({
    'name':[
        'john','mary','peter','jeff','bill'
    ],
    'date_of_birth':[
        '2000-01-01', '1999-12-20', '2000-11-01', '1995-02-25', '1992-06-30',
    ],
})
df

,date_of_birth,name
0,2000-01-01,john
1,1999-12-20,mary
2,2000-11-01,peter
3,1995-02-25,jeff
4,1992-06-30,bill


In [4]:
df.index

RangeIndex(start=0, stop=5, step=1)

## use date column as index

In [5]:
datetime_series = pd.to_datetime(df['date_of_birth'])

datetime_index = pd.DatetimeIndex(datetime_series.values)

df2=df.set_index(datetime_index)

df2.drop('date_of_birth',axis=1,inplace=True)

df2

,name
2000-01-01,john
1999-12-20,mary
2000-11-01,peter
1995-02-25,jeff
1992-06-30,bill


In [6]:
df2.index

DatetimeIndex(['2000-01-01', '1999-12-20', '2000-11-01', '1995-02-25',
               '1992-06-30'],
              dtype='datetime64[ns]', freq=None)

## fill empty periods

In [40]:
df = pd.DataFrame({
    'name':[
        'john','mary','peter','jeff','bill'
    ],
    'year_born':[
        '2000', '1999', '2001', '1995', '1992',
    ],
})
df

,name,year_born
0,john,2000
1,mary,1999
2,peter,2001
3,jeff,1995
4,bill,1992


In [53]:
df.index

RangeIndex(start=0, stop=5, step=1)

In [41]:
datetime_series = pd.to_datetime(df['year_born'])

datetime_index = pd.DatetimeIndex(datetime_series.values)

df3=df.set_index(datetime_index)

df3.drop('year_born',axis=1,inplace=True)

df3.sort_index(inplace=True)

df3

,name
1992-01-01,bill
1995-01-01,jeff
1999-01-01,mary
2000-01-01,john
2001-01-01,peter


In [45]:
df3.index

DatetimeIndex(['1992-01-01', '1995-01-01', '1999-01-01', '2000-01-01',
               '2001-01-01'],
              dtype='datetime64[ns]', freq=None)

In [51]:
df4=df3.asfreq('YS')
df4

,name
1992-01-01,bill
1993-01-01,NaN
1994-01-01,NaN
1995-01-01,jeff
1996-01-01,NaN
1997-01-01,NaN
1998-01-01,NaN
1999-01-01,mary
2000-01-01,john
2001-01-01,peter


In [52]:
df4.index

DatetimeIndex(['1992-01-01', '1993-01-01', '1994-01-01', '1995-01-01',
               '1996-01-01', '1997-01-01', '1998-01-01', '1999-01-01',
               '2000-01-01', '2001-01-01'],
              dtype='datetime64[ns]', freq='AS-JAN')